In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr
from src.DataLoader import TableDatasetPath, TableDatasetDF
from functools import partial
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from src.tools import *
from src.Focal_loss import *
from src.trainer import Trainer
from sklearn.preprocessing import StandardScaler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 256
INIT_PARAM = 1024
NUM_FEATURES = 80

In [3]:
df = pd.read_csv('../data/data_fe.csv')

feature_importance = pd.read_csv('../data/feature_imporstance_WS.csv')

features = list(feature_importance.iloc[:NUM_FEATURES]['feature_names'])
df = df[features + ['Machine failure']]

X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Machine failure']),
                                                 df['Machine failure'],
                                                 shuffle=True,
                                                 stratify=df['Machine failure'])

In [4]:
X_train = pd.DataFrame(X_train).reset_index(drop = True)
X_test = pd.DataFrame(X_test).reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [5]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis = 1)

train_dataset = TableDatasetDF(df_train)
val_dataset = TableDatasetDF(df_test)

In [6]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [7]:
model1 = nn.Sequential(
    nn.Linear(NUM_FEATURES, INIT_PARAM),
    nn.BatchNorm1d(INIT_PARAM),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(INIT_PARAM, int(INIT_PARAM/2)),
    nn.BatchNorm1d(int(INIT_PARAM/2)),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/2), int(INIT_PARAM/4)),
    nn.BatchNorm1d(int(INIT_PARAM/4)),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/4), int(INIT_PARAM/8)),
    nn.BatchNorm1d(int(INIT_PARAM/8)),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/8), int(INIT_PARAM/16)),
    nn.BatchNorm1d(int(INIT_PARAM/16)),
    nn.Dropout(0.1),
    nn.ReLU(),
    ####################
    nn.Linear(int(INIT_PARAM/16), int(INIT_PARAM/32)),
    nn.BatchNorm1d(int(INIT_PARAM/32)),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/32), int(INIT_PARAM/64)),
    nn.BatchNorm1d(int(INIT_PARAM/64)),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/64), int(INIT_PARAM/128)),
    nn.BatchNorm1d(int(INIT_PARAM/128)),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/128), 2)
)

model2 = nn.Sequential(
    nn.Linear(NUM_FEATURES, INIT_PARAM),
    nn.BatchNorm1d(INIT_PARAM),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(INIT_PARAM, int(INIT_PARAM/2)),
    nn.BatchNorm1d(int(INIT_PARAM/2)),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/2), int(INIT_PARAM/4)),
    nn.BatchNorm1d(int(INIT_PARAM/4)),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/4), int(INIT_PARAM/8)),
    nn.BatchNorm1d(int(INIT_PARAM/8)),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/8), int(INIT_PARAM/16)),
    nn.BatchNorm1d(int(INIT_PARAM/16)),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(int(INIT_PARAM/16), 2)
)

models = [model1,model2]

In [8]:
make_experiments(train_dl, val_dl, models, [128])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


Let's see how good I am...: 100%|██████████| 10/10 [00:00<00:00, 15.97it/s]


OSError: [WinError 1314] Клиент не обладает требуемыми правами: 'd:\\Dmitry\\Documents\\GitHub\\Internship-project\\logs\\nn_models\\fe_1024_128_80.pth' -> '../logs/wandb\\run-20230423_205224-22zmh2m2\\files\\nn_models\\fe_1024_128_80.pth'

In [13]:
loss = Loss_class(FocalLoss(gamma=3))
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)
optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

wandb_init_params = dict(
    name=f'fe_{INIT_PARAM}_{BATCH_SIZE}_{NUM_FEATURES}',
    project="Internship_project",
    dir = '../logs/'
)

In [14]:
trainer = Trainer(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/'
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [15]:
trainer.train_model(learning_params)
wandb.finish()

I'm studying hard now🧐, don't disturb!:  97%|█████████▋| 29/30 [00:03<00:00, 13.69it/s]c:\Users\Dmitry\miniconda3\envs\env1\lib\site-packages\sklearn\metrics\_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
Let's see how good I am...: 100%|██████████| 10/10 [00:00<00:00, 18.81it/s]


Epoch: 1 of 40, 0.069 min


Let's see how good I am...: 100%|██████████| 10/10 [00:00<00:00, 18.42it/s]


Epoch: 2 of 40, 0.046 min


Let's see how good I am...: 100%|██████████| 10/10 [00:00<00:00, 18.29it/s]


...

Let's see how good I am...: 100%|██████████| 10/10 [00:00<00:00, 17.97it/s]


Epoch: 14 of 40, 0.047 min


Let's see how good I am...: 100%|██████████| 10/10 [00:00<00:00, 17.00it/s]


Epoch: 15 of 40, 0.049 min


I'm studying hard now🧐, don't disturb!:  73%|███████▎  | 22/30 [00:01<00:00, 14.99it/s]